In [13]:
import pandas as pd
import numpy as np
import os

In [14]:
input_file = r"C:\Users\sharm\OneDrive\Desktop\Kishan\Contractzy\WebScrapping\Tutorial\Financial_Data\MoneyControl\Companies\IT Services & Consulting"

In [ ]:
# company_files = os.listdir(input_file)

# for company in company_files:
#     if company == "Common_Parameters":
#         continue
#     else:
#         folder_path = os.path.join(input_file, company)
        
#         company_folders = os.listdir(folder_path)
        
#         for folders in company_folders:
#             if folders == "Pruned_Excel":
#                 new_folder_path = os.path.join(folder_path, "Pruned_Excel", "Final_Parameters")
                
#                 excel_files_list = os.listdir(new_folder_path)
                
#                 for excel in excel_files_list:
#                     expected_file = f"{company}_Cleaned_Data_Updated.xlsx"
#                     if excel == expected_file:
#                         excel_file = os.path.join(new_folder_path, expected_file)
                        
#                         dfs = pd.read_excel(excel_file, sheet_name=None)
                        
#                         # print(f"Sheets in {excel_file}: {list(dfs.keys())}")
#                         bfs = dfs.get("Balance_Sheet") 
#                         cfs = dfs.get("Cash_Flow")
#                         pls = dfs.get("Profit_Loss") 
#                         qua = dfs.get("Quarterly")
#                         rat = dfs.get("Ratio")
                        
#                         categories = [("Balance_Sheet", bfs), ("Cash_Flow", cfs), 
#                                   ("Profit_Loss", pls), ("Quarterly", qua), ("Ratio", rat)]
                    
#                         for name, cat in categories:
#                             if cat is not None:  # Ensure the sheet exists before accessing .shape
#                                 print(f"Sheet: {name}, Shape: {cat.shape}")
#                             else:
#                                 print(f"Sheet: {name} not found in {excel_file}")
    


Sheet: Balance_Sheet, Shape: (20, 13)
Sheet: Cash_Flow, Shape: (20, 9)
Sheet: Profit_Loss, Shape: (20, 22)
Sheet: Quarterly, Shape: (45, 20)
Sheet: Ratio, Shape: (20, 18)
Sheet: Balance_Sheet, Shape: (20, 13)
Sheet: Cash_Flow, Shape: (20, 9)
Sheet: Profit_Loss, Shape: (20, 22)
Sheet: Quarterly, Shape: (45, 20)
Sheet: Ratio, Shape: (20, 18)
Sheet: Balance_Sheet, Shape: (10, 13)
Sheet: Cash_Flow, Shape: (10, 9)
Sheet: Profit_Loss, Shape: (10, 22)
Sheet: Quarterly, Shape: (25, 20)
Sheet: Ratio, Shape: (10, 18)
Sheet: Balance_Sheet, Shape: (20, 13)
Sheet: Cash_Flow, Shape: (20, 9)
Sheet: Profit_Loss, Shape: (20, 22)
Sheet: Quarterly, Shape: (45, 20)
Sheet: Ratio, Shape: (20, 18)
Sheet: Balance_Sheet, Shape: (20, 13)
Sheet: Cash_Flow, Shape: (20, 9)
Sheet: Profit_Loss, Shape: (20, 22)
Sheet: Quarterly, Shape: (45, 20)
Sheet: Ratio, Shape: (20, 18)
Sheet: Balance_Sheet, Shape: (5, 13)
Sheet: Cash_Flow, Shape: (5, 9)
Sheet: Profit_Loss, Shape: (5, 22)
Sheet: Quarterly, Shape: (10, 20)
Sheet: 

In [ ]:
company_files = os.listdir(input_file)

for company in company_files:
    if company == "Common_Parameters":
        continue

    folder_path = os.path.join(input_file, company)
    
    if not os.path.isdir(folder_path):
        continue  # Skip if it's not a directory

    company_folders = os.listdir(folder_path)

    for folders in company_folders:
        if folders == "Pruned_Excel":
            new_folder_path = os.path.join(folder_path, "Pruned_Excel", "Final_Parameters")
            
            if not os.path.isdir(new_folder_path):  # Ensure the folder exists
                continue
            
            excel_files_list = os.listdir(new_folder_path)

            for excel in excel_files_list:
                expected_file = f"{company}_Cleaned_Data_Updated.xlsx"
                if excel == expected_file:
                    excel_file = os.path.join(new_folder_path, expected_file)
                    
                    # Read all sheets
                    dfs = pd.read_excel(excel_file, sheet_name=None)
                    
                    # Process each sheet
                    for sheet_name, df in dfs.items():
                        if df is None or df.empty:
                            print(f"Skipping empty sheet: {sheet_name}")
                            continue
                        
                        # Step 1: Replace all 0s with NaN
                        df.replace(0, np.nan, inplace=True)

                        # Step 2 & 3: Fill NaNs with mean if count is less than half, else keep as is
                        for column in df.columns:
                            nan_count = df[column].isna().sum()
                            total_rows = len(df)
                            
                            if nan_count < (total_rows / 2):  # Less than half -> Fill with mean
                                mean_value = df[column].mean(skipna=True)  # Compute mean without NaNs
                                df[column].fillna(mean_value, inplace=True)
                            # If more than half -> Keep NaNs as is

                        # Step 5: Convert remaining NaNs back to 0
                        df.fillna(0, inplace=True)

                        # Save the modified sheet back
                        dfs[sheet_name] = df

                    # Save the modified Excel file
                    output_file = os.path.join(new_folder_path, f"{company}_semi_final.xlsx")
                    with pd.ExcelWriter(output_file, engine='xlsxwriter') as writer:
                        for sheet_name, df in dfs.items():
                            df.to_excel(writer, sheet_name=sheet_name, index=False)

                    print(f"Processed and saved: {output_file}")